# Install packages

In [1]:
%%capture
%%bash
python -m pip install -U pip
python -m pip install vaex-core
python -m pip install vaex-hdf5
python -m pip install -U numpy
python -m pip install -U ipython ipykernel

# Please restart the karnel after installation 

In [1]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os

instance_type = 'c5d4xlarge' # change this
results_bucket = f"s3://xdss-benchmarks/benchmarks" # change this

name = 'vaex'
data_path = 'datasets/taxi_1B.hdf5'
output_file = f'{name}_{instance_type}.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {
    'run':[],
    'duration': [],
    'task': []   
}

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks):
    return pd.DataFrame.from_dict(benchmarks, orient='index').T

def persist():
    gc.collect()
    get_results(benchmarks).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, **kwargs):    
    for i in range(2):
        start_time = time.time()
        ret = f(df, **kwargs)
        benchmarks['duration'].append(time.time()-start_time)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]


!mkdir -p results
!mkdir -p datasets
print(f"We test every benchmark twice and save both results")

We test every benchmark twice and save both results


# Benchmark

In [2]:
import vaex
import numpy as np

data = vaex.open(data_path)
print(f"size: {len(data)} with {len(data.columns)} columns")

size: 1173057927 with 18 columns


In [3]:
def open_file(df=None):
    return vaex.open(data_path)

benchmark(open_file, df=data, name='read_file')

read_file took: 0.005378246307373047


0.005378246307373047

In [4]:
def count(df=None):
    return len(df)

benchmark(count, df=data, name='count')

count took: 1.9073486328125e-06


1.9073486328125e-06

In [5]:
def mean(df):
    return df.fare_amount.mean()

benchmark(mean, df=data, name='mean')

mean took: 0.5522580146789551


0.5522580146789551

In [6]:
def standard_deviation(df):
    return df.fare_amount.std()

benchmark(standard_deviation, df=data, name='standard deviation')

standard deviation took: 1.649454116821289


1.649454116821289

In [7]:
def mean_of_sum(df):
    return (df.fare_amount + df.trip_distance).mean()

benchmark(mean_of_sum, df=data, name='sum columns mean')

sum columns mean took: 0.7300000190734863


0.7300000190734863

In [8]:
# lazy evaulation - instant
def sum_columns(df):
    return (df.fare_amount + df.trip_distance)

benchmark(sum_columns, df=data, name='sum columns')

sum columns took: 8.58306884765625e-06


8.58306884765625e-06

In [9]:
def mean_of_product(df):
    return (df.fare_amount * df.trip_distance).mean()

benchmark(mean_of_product, df=data, name='product columns mean')

product columns mean took: 0.7349269390106201


0.7349269390106201

In [10]:
# lazy evaulation - instant
def product(df):
    return df.fare_amount * df.trip_distance

benchmark(product, df=data, name='product columns')

product columns took: 8.344650268554688e-06


8.344650268554688e-06

In [11]:
def lazy_mean(df):
    df['lazy'] = df.fare_amount * df.trip_distance
    return df['lazy'].mean()
    
benchmark(lazy_mean, df=data, name='lazy evaluation')  

lazy evaluation took: 0.7417764663696289


0.7417764663696289

In [12]:
def value_counts(df):
    return df.passenger_count.value_counts()

benchmark(value_counts, df=data, name='value counts')

value counts took: 0.3198368549346924


0.3198368549346924

In [13]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    df['complicated'] = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return df['complicated'].mean()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean')


arithmetic operation mean took: 10.201130867004395


10.201130867004395

In [14]:
# lazy evaulation - instant
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    df['complicated'] = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return df['complicated']

benchmark(complicated_arithmetic_operation, df=data, name='arithmetic operation')

arithmetic operation took: 0.00422978401184082


0.00422978401184082

In [15]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg({'fare_amount': ['mean', 'std'], 
                                               'tip_amount': ['mean', 'std']
                                              })

benchmark(groupby_statistics, df=data, name='groupby statistics')

groupby statistics took: 5.767553091049194


5.767553091049194

In [16]:
other = groupby_statistics(data)

In [17]:
def join_count(df, other):
    return len(df.join(other=other, on = 'passenger_count', rsuffix = '_right'))

benchmark(join_count, data, name='join count', other=other)

join count took: 1.4503576755523682


1.4503576755523682

In [18]:
def join_data(df, other):
    return df.join(other=other, on = 'passenger_count', rsuffix = '_right')

benchmark(join_data, data, name='join', other=other)

join took: 1.0018935203552246


1.0018935203552246

## Filtered data

Dask is not build to run on filter data like you would normally, so we will apply the same strategy

In [19]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

Prepare filtered data and deleted 1054 MB


In [20]:
def filter_data(df):
    expr_filter = (df.pickup_longitude > long_min)  & (df.pickup_longitude < long_max) & \
                  (df.pickup_latitude > lat_min)    & (df.pickup_latitude < lat_max) & \
                  (df.dropoff_longitude > long_min) & (df.dropoff_longitude < long_max) & \
                  (df.dropoff_latitude > lat_min)   & (df.dropoff_latitude < lat_max)
    return df[expr_filter]

benchmark(filter_data, data, name='filter data')

filter data took: 1.1085867881774902


1.1085867881774902

In [21]:
filtered = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

cleaned 2887 mb


In [22]:
benchmark(mean, filtered, name='filtered mean')
benchmark(standard_deviation, filtered, name='filtered standard deviation')
benchmark(mean_of_sum, filtered, name ='filtered sum columns mean')
benchmark(sum_columns, df=filtered, name='filtered sum columns')
benchmark(mean_of_product, filtered, name ='filtered product columns mean')
benchmark(product, df=filtered, name='filtered product columns')
benchmark(mean_of_complicated_arithmetic_operation, filtered, name='filtered arithmetic operation mean')
benchmark(complicated_arithmetic_operation, filtered, name='filtered arithmetic operation')
benchmark(value_counts, filtered, name ='filtered value counts')
benchmark(groupby_statistics, filtered, name='filtered groupby statistics')
other = groupby_statistics(filtered)
benchmark(join_count, filtered, name='filtered join count', other=other)
benchmark(join_data, filtered, name='filtered join', other=other)
print('vaex')
get_results(benchmarks)

<string>:1: RuntimeWarning: invalid value encountered in greater
<string>:1: RuntimeWarning: invalid value encountered in less


filtered mean took: 0.9049773216247559
filtered standard deviation took: 1.9094150066375732
filtered sum columns mean took: 2.2794041633605957
filtered sum columns took: 8.58306884765625e-06
filtered product columns mean took: 2.415121078491211
filtered product columns took: 8.58306884765625e-06
filtered arithmetic operation mean took: 14.02369213104248
filtered arithmetic operation took: 0.008115530014038086
filtered value counts took: 0.5618457794189453
filtered groupby statistics took: 8.287164211273193
filtered join count took: 1.6753106117248535
filtered join took: 1.1281743049621582
vaex


,run,duration,task
0,1,0.00713778,read_file
1,2,0.00537825,read_file
2,1,3.62396e-05,count
3,2,1.90735e-06,count
4,1,0.619524,mean
5,2,0.552258,mean
6,1,1.65766,standard deviation
7,2,1.64945,standard deviation
8,1,4.23071,sum columns mean
9,2,0.73,sum columns mean
